In [1]:
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, IterableDataset

import os
import time
import numpy as np
from typing import List, Tuple

In [2]:
torch.cuda.is_available()

True

In [35]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 1

INPUT_SIZE = 25 * 3
HIDDEN_SIZE = 1024 // 2
EMBEDDING_INPUT_SIZE = 64

LEARNING_RATE = 0.003 #0.0005
L2_WEIGTH_DECAY = 0.0001
EPOCHS = 10

LABELS = {
    1: 0,
    2: 1,
    3: 2,
    4: 3,
    5: 4,
    6: 5,
    7: 6,
    8: 7,
    9: 8,
    10: 9,
    11: 10,
    13: 11,
    15: 12,
    17: 13,
    19: 14,
    20: 15,
    22: 16,
    23: 17,
    25: 18,
    28: 19,
    29: 20,
    30: 21,
    31: 22,
    32: 23,
    33: 24,
    34: 25,
    35: 26,
    36: 27,
    37: 28,
    38: 29,
    39: 30,
    40: 31,
    41: 32,
    42: 33,
    43: 34,
    44: 35,
    45: 36,
    46: 37,
    47: 38,
    48: 39,
    49: 40,
    50: 41,
    51: 42
}

In [4]:
device

device(type='cuda')

In [ ]:
# TODO
# https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset

In [5]:
class IterableMovementDataset(IterableDataset):
    
    NUMBER_OF_JOINTS = 25
    NUMBER_OF_AXES = 3
    
    def __init__(self, root: str, transforms=None):
        self.root = root
        self.transforms = transforms
        
        self.data_files = list(sorted(os.listdir(self.root)))
        self.file_frames: List[int] = []
        
        self.classes = LABELS
        
        self.loaded_data = dict()
        for file_name in self.data_files:
            with open(os.path.join(self.root, file_name), "r") as f:
                self.loaded_data[file_name] = f.read().rstrip('\n').split('\n')

    def _get_file_length(self, file_data: List[str]):
        header = file_data[0].split()[-1].split("_")
        return int(header[-1])

    def __iter__(self):
        for i, file_name in enumerate(self.data_files):
            #action_file = os.path.join(self.root, file_name)
            #with open(action_file, "r") as f:
            #    data_str = f.read().rstrip('\n').split('\n')
            data_str = self.loaded_data[file_name]
            
            sequence_length = self._get_file_length(data_str)
            
            all_frames = []
            for frame in data_str[2:]:  # first two header lines in the file
                all_frames.append(
                    [triple.split(", ") for triple in frame.split("; ")]
                )
            
            all_frames = np.array(all_frames, dtype=np.float32)
            '''
            frame = np.array(
                [
                    triple.split(", ") for triple in data_str[line_indx].split(";")
                ],
                dtype=np.float32
            )
            '''
            assert all_frames.shape == (sequence_length, self.NUMBER_OF_JOINTS, self.NUMBER_OF_AXES)
    
            # get sequence label
            target = [self.classes[int(data_str[0].split()[-1].split("_")[1])]] * sequence_length
            target = np.array(target)
        
            if self.transforms:
                all_frames = self.transforms(all_frames)

            yield all_frames, target
    
    def __len__(self) -> int:
        return len(self.data_files)

In [6]:
class BiRNN(nn.Module):

    def __init__(self, input_size: int, hidden_size: int, linear_input_size: int, num_classes: int):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = 2

        self.lstm = nn.LSTM(input_size, hidden_size, self.num_layers, batch_first=True, bidirectional=True)
        self.embedding = nn.Linear(self.hidden_size * 2, linear_input_size)
        self.relu = nn.ReLU()
        self.do = nn.Dropout(0.5)
        self.classifier = nn.Linear(linear_input_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)

        # Forward to LSTM
        out, _ = self.lstm(x, (h0, c0))  # output format: (batch_size, seq_length, hidden_size * 2)
        out = self.embedding(out[:, -1, :])
        out = self.relu(out)
        out = self.do(out)
        out = self.classifier(out)
        out = self.sigmoid(out)
        return out

In [58]:
class NewBiRNN(nn.Module):
    
    def __init__(self, input_size: int, lstm_hidden_size: int, embedding_output_size: int, num_classes: int):
        super().__init__()
        self.hidden_size = lstm_hidden_size
        self.num_layers = 2  # bi-LSTM
        
        # Embedding part, from 75 -> 64 size
        self.embedding = nn.Linear(input_size, embedding_output_size)
        self.relu = nn.ReLU()
        
        self.lstm = nn.LSTM(embedding_output_size, lstm_hidden_size, self.num_layers, batch_first=True, bidirectional=True)
        self.do = nn.Dropout(0.5)
        self.classifier = nn.Linear(self.hidden_size * self.num_layers, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        
        # Embedding
        out = self.embedding(x)
        out = self.relu(out)
        
        # Bi-LSTM
        out, _ = self.lstm(out, (h0, c0))
        
        out = self.do(out[:, -1, :])
        out = self.classifier(out)
        return self.sigmoid(out)

In [58]:
# Torch dataloader
class MovementsDataset(Dataset):
    
    NUMBER_OF_JOINTS = 25
    NUMBER_OF_AXES = 3
    
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        
        self.data_files = list(sorted(os.listdir(self.root)))
        self.file_frames: List[int] = []

        self.classes = LABELS
        
        # Load number of frames for every file
        for fn in self.data_files:
            with open(os.path.join(self.root, fn)) as f:
                header = f.readline().split()[-1].split("_")
                
                self.file_frames.append(int(header[-1]))  # last element - number_of_frames
        
    def _get_file_index(self, frame_indx) -> Tuple[int, int]:
        start_indx = frame_indx
        for i, nof in enumerate(self.file_frames):
            if start_indx < nof:
                # print(f"{start_indx} - {i}")
                return i, start_indx
            else:
                start_indx -= nof
        
    def __getitem__(self, indx):
        file_indx, line_indx = self._get_file_index(indx)
        action_file = os.path.join(self.root, self.data_files[file_indx])
        
        with open(action_file, "r") as f:
            data_str = f.read().rstrip('\n').split('\n')
        
        line_indx += 2  # first two header lines in the file   
        frame = np.array([triple.split(", ") for triple in data_str[line_indx].split(";")], dtype=np.float32)
        assert frame.shape == (self.NUMBER_OF_JOINTS, self.NUMBER_OF_AXES)
        
        target = self.classes[int(data_str[0].split()[-1].split("_")[1])]
        
        if self.transforms:
            frame = self.transforms(frame)
        
        return frame, target
    
    def __len__(self) -> int:
        return len(self.data_files)

In [7]:
train_dataset = IterableMovementDataset(
    "../data/cross-subject/train",
    transforms=transforms.ToTensor()
)
test_dataset = IterableMovementDataset(
    "../data/cross-subject/val",
    transforms=transforms.ToTensor()
)

In [8]:
train_loader = DataLoader(train_dataset) #, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset) #, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
device

device(type='cuda')

In [ ]:
# set-up
#  - Adam optimizer *
#  - LR = 0.0005 *
#  - batch = 1 *
#  - L2 weight decay = 0.0001 *
#  - dropout = 0.5 *
#  - 200 epochs
#  - Embedding - 64 *
#  - Hidden-state - 1024 --> halved for Bi-LSTM *

In [65]:
# Training the network
#model = BiRNN(75, HIDDEN_SIZE, LINEAR_INPUT_SIZE, len(train_dataset.classes)).to(device)
model = NewBiRNN(INPUT_SIZE, HIDDEN_SIZE, EMBEDDING_INPUT_SIZE, len(train_dataset.classes)).to(device)

if os.path.exists("model.pth"):
    model.load_state_dict(torch.load('model.pth'))
    print("Model loaded")
else:
    print("Pretrained model not found")

PRINT_STEP = 999
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=L2_WEIGTH_DECAY
)

# Train the model
for epoch in range(30):
    s_time = time.time()
    total_loss = 0.0
    
    for i, (sequence, labels) in enumerate(train_loader, 1):
        #print(sequence.shape)
        #print(labels.shape)
        
        sequence = sequence.reshape(-1, 1, 25 * 3).to(device)
        labels = labels.reshape(-1).to(device)
        
        #print(sequence.shape)
        #print(labels.shape)
        #break

        # forward pass
        outputs = model(sequence)
        
        #print(outputs.shape)
        #print(labels.shape)

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % PRINT_STEP == 0:
            print(f"{i}/{len(train_loader)} -  Epoch [{epoch + 1}/{EPOCHS}], average_loss: {round(total_loss / PRINT_STEP, 4)}")
            total_loss = 0.0
    
    print(f"Evaluation time: {time.time() - s_time}s.")
    #break
    
print("DONE")

Model loaded
999/17311 -  Epoch [1/10], average_loss: 3.3571
1998/17311 -  Epoch [1/10], average_loss: 3.3381
2997/17311 -  Epoch [1/10], average_loss: 3.3388
3996/17311 -  Epoch [1/10], average_loss: 3.3192
4995/17311 -  Epoch [1/10], average_loss: 3.3314
5994/17311 -  Epoch [1/10], average_loss: 3.3417
6993/17311 -  Epoch [1/10], average_loss: 3.3026
7992/17311 -  Epoch [1/10], average_loss: 3.3415
8991/17311 -  Epoch [1/10], average_loss: 3.2882
9990/17311 -  Epoch [1/10], average_loss: 3.3093
10989/17311 -  Epoch [1/10], average_loss: 3.3133
11988/17311 -  Epoch [1/10], average_loss: 3.2932
12987/17311 -  Epoch [1/10], average_loss: 3.353
13986/17311 -  Epoch [1/10], average_loss: 3.2959
14985/17311 -  Epoch [1/10], average_loss: 3.3535
15984/17311 -  Epoch [1/10], average_loss: 3.2695
16983/17311 -  Epoch [1/10], average_loss: 3.2953
Evaluation time: 157.76017332077026s.
999/17311 -  Epoch [2/10], average_loss: 3.3542
1998/17311 -  Epoch [2/10], average_loss: 3.341
2997/17311 -  E

7992/17311 -  Epoch [10/10], average_loss: 3.3359
8991/17311 -  Epoch [10/10], average_loss: 3.2865
9990/17311 -  Epoch [10/10], average_loss: 3.3125
10989/17311 -  Epoch [10/10], average_loss: 3.3075
11988/17311 -  Epoch [10/10], average_loss: 3.2893
12987/17311 -  Epoch [10/10], average_loss: 3.3503
13986/17311 -  Epoch [10/10], average_loss: 3.2936
14985/17311 -  Epoch [10/10], average_loss: 3.3505
15984/17311 -  Epoch [10/10], average_loss: 3.266
16983/17311 -  Epoch [10/10], average_loss: 3.2943
Evaluation time: 153.55688571929932s.
999/17311 -  Epoch [11/10], average_loss: 3.3507
1998/17311 -  Epoch [11/10], average_loss: 3.3407
2997/17311 -  Epoch [11/10], average_loss: 3.3359
3996/17311 -  Epoch [11/10], average_loss: 3.3189
4995/17311 -  Epoch [11/10], average_loss: 3.3271
5994/17311 -  Epoch [11/10], average_loss: 3.3413
6993/17311 -  Epoch [11/10], average_loss: 3.3003
7992/17311 -  Epoch [11/10], average_loss: 3.3382
8991/17311 -  Epoch [11/10], average_loss: 3.2868
9990/17

11988/17311 -  Epoch [19/10], average_loss: 3.2826
12987/17311 -  Epoch [19/10], average_loss: 3.3465
13986/17311 -  Epoch [19/10], average_loss: 3.2953
14985/17311 -  Epoch [19/10], average_loss: 3.3489
15984/17311 -  Epoch [19/10], average_loss: 3.2646
16983/17311 -  Epoch [19/10], average_loss: 3.293
Evaluation time: 154.23165655136108s.
999/17311 -  Epoch [20/10], average_loss: 3.3513
1998/17311 -  Epoch [20/10], average_loss: 3.3349
2997/17311 -  Epoch [20/10], average_loss: 3.3316
3996/17311 -  Epoch [20/10], average_loss: 3.3168
4995/17311 -  Epoch [20/10], average_loss: 3.3264
5994/17311 -  Epoch [20/10], average_loss: 3.3369
6993/17311 -  Epoch [20/10], average_loss: 3.2979
7992/17311 -  Epoch [20/10], average_loss: 3.3361
8991/17311 -  Epoch [20/10], average_loss: 3.2848
9990/17311 -  Epoch [20/10], average_loss: 3.3071
10989/17311 -  Epoch [20/10], average_loss: 3.3069
11988/17311 -  Epoch [20/10], average_loss: 3.2849
12987/17311 -  Epoch [20/10], average_loss: 3.3518
13986

15984/17311 -  Epoch [28/10], average_loss: 3.2665
16983/17311 -  Epoch [28/10], average_loss: 3.2926
Evaluation time: 154.22381401062012s.
999/17311 -  Epoch [29/10], average_loss: 3.3507
1998/17311 -  Epoch [29/10], average_loss: 3.3349
2997/17311 -  Epoch [29/10], average_loss: 3.3326
3996/17311 -  Epoch [29/10], average_loss: 3.3203
4995/17311 -  Epoch [29/10], average_loss: 3.3249
5994/17311 -  Epoch [29/10], average_loss: 3.3375
6993/17311 -  Epoch [29/10], average_loss: 3.2977
7992/17311 -  Epoch [29/10], average_loss: 3.3365
8991/17311 -  Epoch [29/10], average_loss: 3.2871
9990/17311 -  Epoch [29/10], average_loss: 3.3042
10989/17311 -  Epoch [29/10], average_loss: 3.3075
11988/17311 -  Epoch [29/10], average_loss: 3.2901
12987/17311 -  Epoch [29/10], average_loss: 3.3493
13986/17311 -  Epoch [29/10], average_loss: 3.2969
14985/17311 -  Epoch [29/10], average_loss: 3.3498
15984/17311 -  Epoch [29/10], average_loss: 3.2667
16983/17311 -  Epoch [29/10], average_loss: 3.2963
Eval

In [66]:
torch.save(model.state_dict(), 'model.pth')

In [67]:
# Test model
with torch.no_grad():
    steps = 10
    def_dict = {
        "correct": 0,
        "above": 0,
    }
    thresholds = [(round((1 / steps) * (i + 1), 4), def_dict.copy()) for i in range(steps)]
    total = 0

    for i, (sequence, labels) in enumerate(test_loader):
        sequence = sequence.reshape(-1, 1, 25 * 3).to(device)
        labels = labels.reshape(-1).to(device)
        
        outputs = model(sequence)

        for val_th, res_dict in thresholds:
            res_dict["above"] += (outputs.data > val_th).sum().item()
            
            # Correctly predicted
            for record_i, label_i in zip(*torch.where(outputs.data > val_th)):
                if label_i == labels[record_i]:
                    res_dict["correct"] += 1
        
        total += labels.size(0)

        if i % 49 == 0:
            print(f"Processed: [{i}/{len(test_dataset)}]")

# Save model
#torch.save(model.state_dict(), 'model.pth')

Processed: [0/2509]
Processed: [49/2509]
Processed: [98/2509]
Processed: [147/2509]
Processed: [196/2509]
Processed: [245/2509]
Processed: [294/2509]
Processed: [343/2509]
Processed: [392/2509]
Processed: [441/2509]
Processed: [490/2509]
Processed: [539/2509]
Processed: [588/2509]
Processed: [637/2509]
Processed: [686/2509]
Processed: [735/2509]
Processed: [784/2509]
Processed: [833/2509]
Processed: [882/2509]
Processed: [931/2509]
Processed: [980/2509]
Processed: [1029/2509]
Processed: [1078/2509]
Processed: [1127/2509]
Processed: [1176/2509]
Processed: [1225/2509]
Processed: [1274/2509]
Processed: [1323/2509]
Processed: [1372/2509]
Processed: [1421/2509]
Processed: [1470/2509]
Processed: [1519/2509]
Processed: [1568/2509]
Processed: [1617/2509]
Processed: [1666/2509]
Processed: [1715/2509]
Processed: [1764/2509]
Processed: [1813/2509]
Processed: [1862/2509]
Processed: [1911/2509]
Processed: [1960/2509]
Processed: [2009/2509]
Processed: [2058/2509]
Processed: [2107/2509]
Processed: [2

In [63]:
# Calculation of evalution metrics for every threshold
ap_score = 0
for th, values in thresholds:
    old_recall = 0

    recall = values["correct"] / total

    if values["above"] > 0:
        precision = values["correct"] / values["above"]
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        precision = 0
        f1_score = 0

    print("\n" + "-" * 15 + f"{th}" + "-" * 15)
    print(f"Test Precision: {round(100 * precision, 4)}%")
    print(f"Test Recall: {round(100 * recall, 4)}%")
    print(f"Test f1_score: {round(f1_score, 4)}")

    ap_score += (recall - old_recall) * precision
    old_recall = recall

# AP - score
print("-" * 32)
print("-" * 14 + " AP " + "-" * 14)
print(f"AP: {round(ap_score * 100, 4)}%")


---------------0.1---------------
Test Precision: 4.8892%
Test Recall: 62.1731%
Test f1_score: 0.0907

---------------0.2---------------
Test Precision: 5.3056%
Test Recall: 58.6124%
Test f1_score: 0.0973

---------------0.3---------------
Test Precision: 5.5928%
Test Recall: 56.302%
Test f1_score: 0.1017

---------------0.4---------------
Test Precision: 5.8304%
Test Recall: 54.4377%
Test f1_score: 0.1053

---------------0.5---------------
Test Precision: 6.0519%
Test Recall: 52.7792%
Test f1_score: 0.1086

---------------0.6---------------
Test Precision: 6.2751%
Test Recall: 51.1258%
Test f1_score: 0.1118

---------------0.7---------------
Test Precision: 6.5132%
Test Recall: 49.2905%
Test f1_score: 0.1151

---------------0.8---------------
Test Precision: 6.8073%
Test Recall: 47.025%
Test f1_score: 0.1189

---------------0.9---------------
Test Precision: 7.2318%
Test Recall: 43.3316%
Test f1_score: 0.1239

---------------1.0---------------
Test Precision: 0%
Test Recall: 0.0%
Tes

In [ ]:
# TODO
#  - evaluacne metriky
#  - format dat pre trenovanie

In [ ]:
####### Evalution metrics

In [ ]:
# Calculate precision
#  - Precision: the ratio of correctly annotated frames and all the model-annotated frames on test sequences
#  - spravne anotovane (correctly annotates)
#  - zoberiem threshold pre kazdy output v kazdom snimku --> pocet tried do ktorych sa klasifikoval snimok (all model-annotated frames)